In [73]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Circle, Rectangle, Arc
from scipy.stats import pearsonr

In [82]:
shooting_stats = pd.read_csv("/Users/sydnee/Downloads/16th Grade/Data Science/shooting.csv")
lineup_stats = pd.read_csv("/Users/sydnee/Downloads/16th Grade/Data Science/alllineup_stats.csv")
player_stats = pd.read_csv("/Users/sydnee/Downloads/16th Grade/Data Science/player_100stats.csv")

In [15]:
shooting_stats_orig = shooting_stats.copy()
# shooting_stats_orig.to_csv(f'shooting.csv')

In [5]:
# format names to first initial. last name
def format_name(full_name):
    name_parts = full_name.split()
    if len(name_parts) > 1:
        return f"{name_parts[0][0]}. {name_parts[1]}"
    else:
        return full_name
shooting_stats['Player'] = shooting_stats['Player'].apply(format_name)

            Player Team  Age   G    MP    FG%  Dist.   2P_x  0-3_x  3-10_x  \
0        D. Murray  ATL   27  78  2783  0.459   14.9  0.621  0.170   0.191   
1    B. Bogdanović  ATL   31  79  2401  0.428   18.4  0.417  0.123   0.148   
2           S. Bey  ATL   24  63  2062  0.416   14.5  0.486  0.287   0.155   
3         T. Young  ATL   25  54  1942  0.430   17.6  0.535  0.121   0.189   
4       J. Johnson  ATL   22  56  1889  0.511   10.6  0.715  0.342   0.272   
..             ...  ...  ...  ..   ...    ...    ...    ...    ...     ...   
652    T. Vukcevic  WAS   20  10   153  0.433   16.8  0.463  0.209   0.164   
653     J. Bernard  WAS   24  19   149  0.453   14.0  0.547  0.328   0.172   
654     R. Rollins  WAS   21  10    66  0.520    8.4  0.880  0.320   0.360   
655      H. Diallo  WAS   25   2     5  0.500    0.1  1.000  1.000   0.000   
656     T. Jemison  WAS   24   2     1    NaN    NaN    NaN    NaN     NaN   

     ...  16-3P_y   3P_y   2P_z   3P_z   %FGA  #_x   %3PA    3P

In [6]:
# team abbreviations
teams_dict = {'hawks' : 'ATL',
              'celtics' : 'BOS',
              'nets' : 'BRK',
              'hornets' : 'CHO',
              'bulls' : 'CHI',
              'cavaliers' : 'CLE',
              'mavericks' : 'DAL',
              'nuggets' : 'DEN',
              'pistons' : 'DET',
              'warriors' : 'GSW',
              'rockets' : 'HOU',
              'pacers' : 'IND',
              'clippers' : 'LAC',
              'lakers' : 'LAL',
              'grizzlies' : 'MEM',
              'heat' : 'MIA',
              'bucks' : 'MIL',
              'timberwolves' : 'MIN',
              'pelicans' : 'NOP',
              'knicks' : 'NYK',
              'thunder' : 'OKC',
              'magic' : 'ORL',
              '76ers' : 'PHI',
              'suns' : 'PHO',
              'trailblazers' : 'POR',
              'kings' : 'SAC',
              'spurs' : 'SAS',
              'raptors' : 'TOR',
              'jazz' : 'UTA',
              'wizards' : 'WAS'}

In [8]:
player_in_lineup = (shooting_stats['Player'].isin(lineup_stats['Player 1'])|shooting_stats['Player'].isin(lineup_stats['Player 2']))
shooting_stats = shooting_stats[player_in_lineup]
shooting_stats.reset_index(drop=True, inplace=True)

In [20]:
print(pd.unique(pd.concat([lineup_stats['Player 1'], lineup_stats['Player 2']])).shape)
print(pd.unique(shooting_stats['Player']).shape)
#shooting_stats.to_csv('/Users/sydnee/Downloads/16th Grade/Data Science/shooting.csv', index=False)

(268,)
(268,)


In [21]:
unique_lineup_players = pd.unique(pd.concat([lineup_stats['Player 1'], lineup_stats['Player 2']]))
player_stats_names = shooting_stats['Player'].unique()
players_not_in_stats = unique_lineup_players[~pd.Series(unique_lineup_players).isin(player_stats_names)]
players_not_in_stats_list = players_not_in_stats.tolist()
print(players_not_in_stats_list)

[]


In [22]:
name_counts = shooting_stats['Player'].value_counts()
duplicate_names = name_counts[name_counts > 1]
print(duplicate_names.index.tolist())

['P. Siakam', 'G. Williams', 'I. Quickley', 'K. Olynyk', "R. O'Neale", 'D. Schroder', 'T. Rozier', 'P. Washington', 'B. Hield']


In [62]:
def calculate_diff(player_1, player_2, stat_columns):
    stats_1 = shooting_stats[shooting_stats['Player'] == player_1].iloc[0]
    stats_2 = shooting_stats[shooting_stats['Player'] == player_2].iloc[0]
    differences = {}
    for col in stat_columns:
        diff = abs(stats_1[col] - stats_2[col])
        differences[f"{col}_diff"] = diff
    return differences

shooting_stat_columns = ['Dist.', '0-3_x', '3-10_x', '10-16_x', '16-3P_x', '3P_x']

differences_list = []

for index, row in lineup_stats.iterrows():
    player_1 = row['Player 1']
    player_2 = row['Player 2']
    shooting_diff = calculate_diff(player_1, player_2, shooting_stat_columns)
    differences_list.append(shooting_diff)

differences_df = pd.DataFrame(differences_list)
lineup_stats = pd.concat([lineup_stats, differences_df], axis=1)


lineup_stats.to_csv('/Users/sydnee/Downloads/16th Grade/Data Science/alllineup_stats.csv', index=False)

print(differences_df)


     Dist._diff  0-3_x_diff  3-10_x_diff  10-16_x_diff  16-3P_x_diff  \
0          12.4       0.523        0.100         0.170         0.076   
1           0.4       0.117        0.036         0.148         0.068   
2           3.5       0.047        0.043         0.086         0.028   
3           1.2       0.004        0.042         0.044         0.010   
4           4.3       0.172        0.081         0.098         0.060   
..          ...         ...          ...           ...           ...   
591         7.7       0.297        0.022         0.003         0.023   
592         3.9       0.157        0.008         0.081         0.036   
593         2.3       0.023        0.110         0.009         0.007   
594         2.9       0.089        0.148         0.136         0.024   
595         0.1       0.047        0.014         0.035         0.057   

     3P_x_diff  
0        0.377  
1        0.135  
2        0.204  
3        0.080  
4        0.094  
..         ...  
591      0.293  

In [66]:
top_20_dist = lineup_stats[['Player 1', 'Player 2', 'Team', 'MP', 'ORTG2', 'DRTG2', 'Dist._diff']].nlargest(20, 'Dist._diff')
top_20_dist['Rank'] = range(1, len(top_20_dist) + 1)
top_20_dist = top_20_dist[['Rank', 'Player 1', 'Player 2', 'Team', 'MP', 'ORTG2', 'DRTG2', 'Dist._diff']]
top_20_dist = top_20_dist.reset_index(drop=True)
display(Markdown("## Top 20 Distance"))
display(top_20_dist)

## Top 20 Distance

,Rank,Player 1,Player 2,Team,MP,ORTG2,DRTG2,Dist._diff
0,1,T. Jackson-Davis,K. Thompson,GSW,643:35:00,125.586507,114.716564,17.8
1,2,T. Hardaway,D. Lively,DAL,601:51:00,120.699134,116.270202,17.3
2,3,L. Kornet,P. Pritchard,BOS,726:42:00,142.345440,112.616886,17.0
3,4,T. Murphy,Z. Williamson,NOP,925:55:00,122.197397,113.399675,16.3
4,5,K. Looney,K. Thompson,GSW,821:11:00,122.665891,115.698597,16.1
5,6,M. Conley,R. Gobert,MIN,1805:05:00,129.895385,107.866152,16.1
6,7,Bogdan Bogdanovic,C. Capela,ATL,977:24:00,121.728762,117.613016,15.9
7,8,D. DiVincenzo,I. Hartenstein,NYK,1327:48:00,126.295785,111.032567,15.7
8,9,S. Curry,K. Looney,GSW,850:27:00,125.312127,115.316766,15.6
9,10,C. Capela,T. Young,ATL,936:35:00,121.851257,118.912837,15.1


In [67]:
top_20_dist = lineup_stats[['Player 1', 'Player 2', 'Team', 'MP', 'ORTG2', 'DRTG2', 'Dist._diff']].nlargest(20, 'ORTG2')
top_20_dist['Rank'] = range(1, len(top_20_dist) + 1)
top_20_dist = top_20_dist[['Rank', 'Player 1', 'Player 2', 'Team', 'MP', 'ORTG2', 'DRTG2', 'Dist._diff']]
top_20_dist = top_20_dist.reset_index(drop=True)
display(Markdown("## Top 20 Offensive Ratings"))
display(top_20_dist)

## Top 20 Offensive Ratings

,Rank,Player 1,Player 2,Team,MP,ORTG2,DRTG2,Dist._diff
0,1,L. Kornet,P. Pritchard,BOS,726:42:00,142.345440,112.616886,17.0
1,2,D. Gafford,Tyus Jones,WAS,1073:37:00,131.320259,117.614381,11.6
2,3,A. Horford,D. White,BOS,1095:11:00,129.973703,111.558477,0.8
3,4,M. Conley,R. Gobert,MIN,1805:05:00,129.895385,107.866152,16.1
4,5,Terance Mann,I. Zubac,LAC,1104:42:00,129.662039,115.805554,9.4
5,6,S. Bey,O. Okongwu,ATL,890:52:00,129.141365,119.476439,7.7
6,7,T. Haliburton,O. Toppin,IND,882:34:00,129.000000,119.000000,4.9
7,8,K. Irving,D. Lively,DAL,611:46:00,128.842862,114.389915,12.7
8,9,A. Gordon,N. Jokic,DEN,1990:12:00,128.589801,110.410199,2.3
9,10,J. Harden,I. Zubac,LAC,1365:52:00,128.570497,114.469787,14.8


In [69]:
top_20_dist = lineup_stats[['Player 1', 'Player 2', 'Team', 'MP', 'ORTG2', 'DRTG2', 'Dist._diff']].nsmallest(20, 'DRTG2')
top_20_dist['Rank'] = range(1, len(top_20_dist) + 1)
top_20_dist = top_20_dist[['Rank', 'Player 1', 'Player 2', 'Team', 'MP', 'ORTG2', 'DRTG2', 'Dist._diff']]
top_20_dist = top_20_dist.reset_index(drop=True)
display(Markdown("## Top 20 Defensive Ratings"))
display(top_20_dist)

## Top 20 Defensive Ratings

,Rank,Player 1,Player 2,Team,MP,ORTG2,DRTG2,Dist._diff
0,1,R. Gobert,N. Reid,MIN,918:35:00,117.413553,106.037967,11.8
1,2,K. Anderson,R. Gobert,MIN,865:48:00,121.008022,106.938817,6.3
2,3,R. Gobert,K. Towns,MIN,1279:35:00,120.587085,107.254036,10.0
3,4,K. Anderson,N. Reid,MIN,1117:45:00,112.206870,107.804580,5.5
4,5,M. Conley,R. Gobert,MIN,1805:05:00,129.895385,107.866152,16.1
5,6,C. Anthony,J. Isaac,ORL,713:36:00,117.020712,107.991124,2.3
6,7,A. Edwards,R. Gobert,MIN,1922:53:00,118.239230,108.029717,11.4
7,8,N. Alexander-Walker,R. Gobert,MIN,1064:49:00,121.653446,108.089117,15.0
8,9,R. Gobert,J. McDaniels,MIN,1577:35:00,118.488660,108.351770,10.9
9,10,G. Bitadze,J. Suggs,ORL,655:44:00,121.116333,108.376733,12.9


In [79]:
corr_coefficient_dist, _ = pearsonr(lineup_stats['Dist._diff'], lineup_stats['ORTG2'])
corr_coefficient_03, _ = pearsonr(lineup_stats['0-3_x_diff'], lineup_stats['ORTG2'])
corr_coefficient_310, _ = pearsonr(lineup_stats['3-10_x_diff'], lineup_stats['ORTG2'])
corr_coefficient_1016, _ = pearsonr(lineup_stats['10-16_x_diff'], lineup_stats['ORTG2'])
corr_coefficient_163P, _ = pearsonr(lineup_stats['16-3P_x_diff'], lineup_stats['ORTG2'])
corr_coefficient_3P, _ = pearsonr(lineup_stats['3P_x_diff'], lineup_stats['ORTG2'])
print(corr_coefficient_dist)
print(corr_coefficient_03)
print(corr_coefficient_310)
print(corr_coefficient_1016)
print(corr_coefficient_163P)
print(corr_coefficient_3P)

0.34917811049064706
0.32060558101550096
0.11414147850930953
0.02532513327541166
0.00978150223097169
0.3338291577836093


In [80]:
corr_coefficient_dist, _ = pearsonr(lineup_stats['Dist._diff'], lineup_stats['DRTG2'])
corr_coefficient_03, _ = pearsonr(lineup_stats['0-3_x_diff'], lineup_stats['DRTG2'])
corr_coefficient_310, _ = pearsonr(lineup_stats['3-10_x_diff'], lineup_stats['DRTG2'])
corr_coefficient_1016, _ = pearsonr(lineup_stats['10-16_x_diff'], lineup_stats['DRTG2'])
corr_coefficient_163P, _ = pearsonr(lineup_stats['16-3P_x_diff'], lineup_stats['DRTG2'])
corr_coefficient_3P, _ = pearsonr(lineup_stats['3P_x_diff'], lineup_stats['DRTG2'])
print(corr_coefficient_dist)
print(corr_coefficient_03)
print(corr_coefficient_310)
print(corr_coefficient_1016)
print(corr_coefficient_163P)
print(corr_coefficient_3P)

-0.23182694689534816
-0.16737784103077075
-0.2124079006691456
-0.05417686226675149
-0.042733706933496346
-0.26553327345640687


In [89]:
def usage_diff(player_1, player_2, stat_columns):
    stats_1 = player_stats[player_stats['Name'] == player_1].iloc[0]
    stats_2 = player_stats[player_stats['Name'] == player_2].iloc[0]
    differences = {}
    for col in stat_columns:
        diff = abs(stats_1[col] - stats_2[col])
        differences[f"{col}_diff"] = diff
    return differences

shooting_stat_columns = ['USG%']

differences_list = []

for index, row in lineup_stats.iterrows():
    player_1 = row['Player 1']
    player_2 = row['Player 2']
    shooting_diff = usage_diff(player_1, player_2, shooting_stat_columns)
    differences_list.append(shooting_diff)

differences_df = pd.DataFrame(differences_list)
lineup_stats = pd.concat([lineup_stats, differences_df], axis=1)


#lineup_stats.to_csv('/Users/sydnee/Downloads/16th Grade/Data Science/alllineup_stats.csv', index=False)

print(differences_df)

     USG%_diff
0     3.701687
1    10.914911
2     0.660310
3     2.252758
4     8.616759
..         ...
591   6.926140
592  16.519254
593   6.940771
594   9.423601
595  20.050257

[596 rows x 1 columns]


In [90]:
corr_coefficient_usg, _ = pearsonr(lineup_stats['USG%_diff'], lineup_stats['ORTG2'])
print(corr_coefficient_usg)
corr_coefficient_usg, _ = pearsonr(lineup_stats['USG%_diff'], lineup_stats['DRTG2'])
print(corr_coefficient_dist)

0.05075382165994584
-0.23182694689534816


In [94]:
print(lineup_stats)
lineup_stats.to_csv(f'alllineup_stats.csv')

              Player 1      Player 2 Team          MP   PTS   FG  FGA    FG%  \
0            C. Capela     D. Murray  ATL  1771:30:00  -1.1 -0.7  5.0 -0.033   
1               S. Bey     D. Murray  ATL  1594:50:00  -4.1 -2.8  2.0 -0.041   
2    Bogdan Bogdanovic     D. Murray  ATL  1427:54:00   1.4  0.1  4.5 -0.023   
3            D. Hunter     D. Murray  ATL  1413:14:00  -1.5 -1.4  1.5 -0.024   
4           J. Johnson     D. Murray  ATL  1262:33:00  -3.8 -2.4  0.6 -0.029   
..                 ...           ...  ...         ...   ...  ...  ...    ...   
591          D. Avdija    D. Gafford  WAS   771:47:00  -5.4 -0.2 -3.2  0.015   
592       B. Coulibaly      J. Poole  WAS   718:57:00 -19.8 -8.3 -4.1 -0.071   
593          D. Avdija  B. Coulibaly  WAS   694:40:00 -12.1 -5.7 -4.6 -0.039   
594         Tyus Jones    C. Kispert  WAS   607:29:00 -15.0 -6.0 -4.3 -0.042   
595         C. Kispert     L. Shamet  WAS   420:09:00 -10.2 -4.5 -0.2 -0.049   

      3P    3PA  ...   PF       ORTG2  

In [93]:
from sklearn.ensemble import RandomForestRegressor

X = lineup_stats[['Dist._diff', '0-3_x_diff', '3-10_x_diff', '10-16_x_diff', '16-3P_x_diff', '3P_x_diff', 'USG%_diff']]
y = lineup_stats['ORTG2']

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)

feature_importances = rf.feature_importances_
normalized_importances = feature_importances / sum(feature_importances)

for feature, importance in zip(X.columns, normalized_importances):
    print(f'Feature: {feature}, Weight: {importance}')

Feature: Dist._diff, Weight: 0.12656398752325762
Feature: 0-3_x_diff, Weight: 0.20031029938764383
Feature: 3-10_x_diff, Weight: 0.14337630194988518
Feature: 10-16_x_diff, Weight: 0.11114027128476078
Feature: 16-3P_x_diff, Weight: 0.12033389883189363
Feature: 3P_x_diff, Weight: 0.15802406886639037
Feature: USG%_diff, Weight: 0.1402511721561685


In [ ]:
# compatibility score
